In [ ]:
from google.colab import drive
import torch
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as T
import os
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/PROYECTO FINAL/dataset'
os.chdir(DATA_PATH)

## LIMPIAR FONDO

In [ ]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dinov2_vits14.to(device)
dinov2_vits14.eval()

In [ ]:
# Transformación de entrada
transform = T.Compose([
    T.Resize((518, 518)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

def limpiar_fondo_dino(img_path, output_path, threshold=0.1, alpha=True, blur_size=5, visualize=False):
    """
    Elimina el fondo de una imagen usando DINOv2.
    threshold: umbral para la máscara (0.05-0.2 recomendado)
    blur_size: tamaño del kernel Gaussiano para suavizar la máscara
    visualize: si True muestra el mapa de activación
    """
    pil_img = Image.open(img_path).convert("RGB")
    orig_img = np.array(pil_img)

    # Preprocesar
    inp = transform(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = dinov2_vits14(inp)

    # Mapa de activación promedio por canal
    activation_map = feats.mean(1).squeeze().cpu().numpy()
    activation_map = np.nan_to_num(activation_map, nan=0.0, posinf=1.0, neginf=0.0)

    # Normalizar
    if activation_map.max() - activation_map.min() != 0:
        activation_map = (activation_map - activation_map.min()) / (activation_map.max() - activation_map.min())
    else:
        activation_map = np.zeros_like(activation_map)

    # Suavizar para no perder pequeñas partes del ala
    activation_map = cv2.GaussianBlur(activation_map, (blur_size, blur_size), 0)

    # Redimensionar al tamaño original
    activation_map = cv2.resize(activation_map, (orig_img.shape[1], orig_img.shape[0]))

    # Visualizar mapa de activación (opcional)
    if visualize:
        plt.imshow(activation_map, cmap='hot')
        plt.colorbar()
        plt.show()

    # Crear máscara binaria
    mask = (activation_map > threshold).astype(np.uint8)

    # Aplicar máscara
    if alpha:
        result = cv2.cvtColor(orig_img, cv2.COLOR_RGB2BGRA)
        result[:, :, 3] = mask * 255
    else:
        result = orig_img.copy()
        result[mask == 0] = 255

    # Guardar imagen
    cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGRA) if alpha else cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

In [ ]:
input_dir = "/content/drive/MyDrive/PROYECTO FINAL/dataset/alas_png"
output_dir = "/content/drive/MyDrive/PROYECTO FINAL/dataset/alas_limpias"
os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(input_dir):
    if file.lower().endswith(".png"):
        in_path = os.path.join(input_dir, file)
        out_path = os.path.join(output_dir, file)
        limpiar_fondo_dino(in_path, out_path, threshold=0.1, alpha=True, blur_size=5, visualize=False)

print("Todas las alas fueron procesadas correctamente con fondo transparente.")